In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Link to download the Chrome webdriver

# https://chromedriver.chromium.org/downloads

In [ ]:
driver = webdriver.Chrome('/Users/anthonyvalle/Downloads/Chrome/chromedriver')

# This link is a webpage filled with other links that direct you to data in the form of html tables

In [ ]:
url = 'http://www.nuforc.org/webreports/ndxpost.html'

#  The variable 'links' is going to cointain all the 'a' tags on the wepage

In [ ]:
driver.get(url)
links = driver.find_elements_by_tag_name('a')

# For each element in the variable 'links', which is a list of all 'a' tag elements, we append the text of the actual tag to a list called link_lists

# The first 'a' tag within that webpage has a link that is not useful therfore it is popped off

In [ ]:
link_list = []
for i in links:
    link_list.append(i.text)
link_list.pop(0)

# The function I am going to use to scrape all the data when each 'a' tag is clicked from the list of links

In [ ]:
def scraperows():
    # Create an empty list
    rows=[]
    
    # Gather all 'td' tags in the html table, essentially each table cell
    body = driver.find_elements_by_tag_name('td')
    
    # A counter that will help in grouping a certain number of cells to represent a row
    # This is important because we are scrapinf indivdual cells, not rows
    counter = 0
    
    # A temporary list that will hold all the cells of one indvidual row
    templist = []
    
    for elements in body:
        
        # Gather the text of the 'td' tag
        text = elements.text
        
        # Append that text into a temporary list
        templist.append(text)
        
        # Counter increment
        counter += 1
        
        # If the counter equals 7 then we have scraped all the cells of an individual row and must seperate it
        if counter == 7:
            
            #Reset the Counter
            counter = 0
            
            # Append a copy of the temporary list to the list 'rows'
            # We must append a copy becuase the list will be cleared which will clear it within the list 'rows'
            rows.append(templist.copy())
            
            # Clearing the temporary list
            templist.clear()
    
    # Procedure to seperate the time and date since they are in the same cell
    # At this point in time the list 'rows' is filled with individual rows of all the cells we scraped
    for num, elements in enumerate(rows):
        
        # Gather the element with the date and time which is always the first element
        date_time = rows[num][0]
        
        # Split the date and time
        split = date_time.split(' ')
        
        # Remove the concatenated date and time
        rows[num].remove(date_time)
        
        # Insert the indivdual date and time into the row
        for i in split:

            rows[num].insert(0, i)
            
    return(rows)

        

# Testing the function

In [ ]:
rows = scraperows()

# The scraping procedure

In [ ]:
# the list 'all_rows' will be the final list of ALL scraped rows
all_rows = []

# For each element in the list of the link texts
for i in link_list:
    
    time.sleep(.5)
    
    # Find the link by link text
    link = driver.find_element_by_link_text(i)
    
    # Click the link
    link.click()
    
    # Scrape the html table and assign it to 'rows'
    rows = scraperows()
    
    # Append every scraped row into the final list from line 1, 'all_rows'
    for lists in rows:
        
        all_rows.append(lists)
    
    # Clear 'rows'
    rows.clear()

    time.sleep(.5)
    
    # Go back to the main webpage with all the links anmd repeat
    driver.back()

    

# After 2 hours of scraping the code mysteriously failed but I was able to scrape about 40,000 rows of data

# Convert 'all_rows' to a pandas DataFrame

In [ ]:
data = pd.DataFrame(all_rows)

# Drop two odd columns that contained nothing

In [ ]:
data.drop([8,9], axis=1, inplace=True)

In [ ]:
data.head(1)

# Save the DataFrame to a csv

In [ ]:
data.to_csv('UFO.csv')

# After the scraper failed I found the date of the link where it left off
# Using this date I found the index of the link where the scraper failed

In [ ]:
for i in link_list:
    if i == '05/29/2011':
        print(link_list.index(i))

# Using the index I created a new list of the links with only the links that werent scraped and continued to scrape my data 

In [ ]:
newlist = link_list[370::]

# Same process as before

In [ ]:
new_all_rows = []

for i in newlist:
        
    time.sleep(.5)
    link = driver.find_element_by_link_text(i)
    link.click()
    
    rows = scraperows()
    
    for lists in rows:
        
        new_all_rows.append(lists)
      
    rows.clear()

    time.sleep(.5)

    driver.back()

# After 2 more hours of scraping I was able to get about 65,000 more rows of data

In [ ]:
len(new_all_rows)

In [ ]:
newdata =pd.DataFrame(new_all_rows)

In [ ]:
newdata.drop([8,9,10,11,12], axis=1, inplace=True)

In [ ]:
newdata.to_csv('NewUFO.csv')